# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

Framework : 애플리케이션 개발 시 필수적인 코드, 알고리즘, DB연동과 같은 기능들을 위해 어느정도의 구조를 갖춘 도구  
Library : sw를 개발할 때 컴퓨터 프로그램이 사용하는 비휘발성 자원의 모임, 특정 기능을 모아둔 코드, 함수들의 집합

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

머신러닝 : 사람이 정한 모델과 특징 추출 방법을 사용하여 데이터를 기반으로 학습해 추론할 수 있게 하는 기술  
딥러닝 : 신경망을 통해 인공지능을 만드는 머신러닝의 한 종류로 Multi-layer-perceptron의 방식으로 구현됨


### Q3. 아래의 코드에 주석 달기.

In [9]:
#library and device setting 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

device = torch.device('mps')
torch.manual_seed(45)
if device == 'mps':
    torch.cuda.manual_seed_all(45)
print(device)

mps


In [10]:
#hyperparameter setting
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [11]:
#torchvision에서 MNIST dataset 불러오기 
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() #tensor로 변환 
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

100%|██████████| 9912422/9912422 [00:01<00:00, 9873807.71it/s] 


Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 6693319.36it/s]


Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5129273.21it/s]


Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 975249.76it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



In [12]:
#batch 학습을 위한 DataLoader setting 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

#data 미리보기 
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

In [13]:
#network setting
class ConvNet(nn.Module): #torch.module을 상속받아 convolution network만들기 
  def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)        #convolution network 1 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)       #convolution network 2
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)   #dropout 
        self.mp = nn.MaxPool2d(2)                           #maxpooling
        self.fc1 = nn.Linear(320,100)                       #fc-layer
        self.fc2 = nn.Linear(100,10)                        #fc-layer


  def forward(self, x): #forward propagation을 위한 network 
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)

In [14]:
#model instance
model = ConvNet().to(device)
#loss function setting 
criterion = nn.CrossEntropyLoss().to(device)
#optimizer setting 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [15]:
#train 
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader: #train_loader로 부터 batch 단위로 data 받기
        data = data.to(device)        #data -> mps
        target = target.to(device)    #target -> mps
        optimizer.zero_grad()         #weight initialization
        hypothesis = model(data)      
        cost = criterion(hypothesis, target) #loss function 
        cost.backward()                      #back propagation
        optimizer.step()                     #weight 갱신
        avg_cost += cost / len(train_loader) #average cost계산
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_4876/3803583406.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


[Epoch:    1]  cost = 0.329498947
[Epoch:    2]  cost = 0.110761248
[Epoch:    3]  cost = 0.0862471685
[Epoch:    4]  cost = 0.0730262101
[Epoch:    5]  cost = 0.0654386058


In [16]:
#test 
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_4876/3803583406.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Test Accuracy:  98.64 %
